In [ ]:
!pip install gradio sentence-transformers faiss-cpu datasets


In [ ]:
import pandas as pd

# Load the dataset from HuggingFace
df = pd.read_json("https://huggingface.co/datasets/Abirate/english_quotes/resolve/main/quotes.jsonl", lines=True)

# Basic cleaning
df = df.dropna(subset=['quote', 'author', 'tags'])
df['text'] = df['quote'] + " - " + df['author'] + " [" + df['tags'].apply(lambda x: ', '.join(x)) + "]"
df.head(2)


,quote,author,tags,text
0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"[be-yourself, gilbert-perreira, honesty, inspi...",“Be yourself; everyone else is already taken.”...
1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"[best, life, love, mistakes, out-of-control, t...","“I'm selfish, impatient and a little insecure...."


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode quotes
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)
embeddings = np.array(embeddings).astype("float32")

# Build FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

In [ ]:
def retrieve_quotes(query, top_k=5):
    query_vec = model.encode([query]).astype("float32")
    D, I = index.search(query_vec, top_k)

    results = []
    for idx, score in zip(I[0], D[0]):
        quote = df.iloc[idx]
        results.append({
            "quote": quote['quote'],
            "author": quote['author'],
            "tags": quote['tags'],
            "similarity": round(float(score), 3)
        })
    return results


In [ ]:
# Step 3: RAGAS evaluation
    ragas_data = Dataset.from_dict({
        "question": [user_input],
        "ground_truth": [quote_text],  # original quote
        "contexts": [[context]],
        "answer": [answer]
    })

    scores = {
        "faithfulness": round(faithfulness.compute(ragas_data), 3),
        "answer_relevancy": round(answer_relevancy.compute(ragas_data), 3),
        "context_precision": round(context_precision.compute(ragas_data), 3),
        "context_recall": round(context_recall.compute(ragas_data), 3)
    }

    result = {
        "Retrieved Quote": quote_text,
        "Author": author,
        "Tags": tags,
        "LLM Answer": answer,
        "RAGAS Scores": scores
    }

    return json.dumps(result, indent=2)

In [ ]:
def retrieve_quotes(query, top_k=1):  # Set top_k=1
    query_vec = model.encode([query]).astype("float32")
    D, I = index.search(query_vec, top_k)

    results = []
    for idx, score in zip(I[0], D[0]):
        quote = df.iloc[idx]
        results.append({
            "quote": quote['quote'],
            "author": quote['author'],
            "tags": quote['tags'],
            "similarity": round(float(score), 3)
        })
    return results


In [ ]:
import gradio as gr
import json

'''def query_rag_system(user_input):
    results = retrieve_quotes(user_input)
    return json.dumps(results, indent=2)'''
def query_rag_system(user_input):
    results = retrieve_quotes(user_input, top_k=1)  # Only 1 result
    return json.dumps(results[0], indent=2)

iface = gr.Interface(
    fn=query_rag_system,
    inputs=gr.Textbox(label="Enter your quote query"),
    outputs=gr.Textbox(label="Retrieved Quotes (JSON)"),
    title="RAG-based Quote Retrieval System",
    description="Ask natural language questions like 'quotes about courage by women authors'"
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d236e1caaf38523c37.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio ragas datasets evaluate openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.60
    Uninstalling langchain-core-0.3.60:
      Successfully uninstalled langchain-core-0.3.60


In [ ]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import Dataset
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
import gradio as gr
import requests
import json

# Load dataset
df = pd.read_json("hf://datasets/Abirate/english_quotes/quotes.jsonl", lines=True)

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare FAISS index
embeddings = model.encode(df['quote'].tolist(), show_progress_bar=True).astype('float32')
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# OpenRouter: Mistral API
API_KEY = "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"  # Replace with your OpenRouter API key
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

In [ ]:
def query_rag_with_eval(user_input):
    # Step 1: Retrieve 1 quote
    query_vec = model.encode([user_input]).astype("float32")
    D, I = index.search(query_vec, 1)
    quote = df.iloc[I[0][0]]

    quote_text = quote['quote']
    author = quote['author']
    tags = quote['tags']
    context = f"{quote_text} - {author} [{', '.join(tags)}]"

    # Step 2: Generate answer with Mistral (OpenRouter)
    prompt = f"Answer the question based on the quote context below.\n\nQuestion: {user_input}\n\nContext: {context}\n\nAnswer:"
    data = {
        "model": "mistral/mistral-7b-instruct",  # or openchat, llama2 etc.
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
    try:
        answer = response.json()['choices'][0]['message']['content']
    except:
        answer = "LLM response error. Check your API key or quota."

    # Step 3: RAGAS evaluation
    ragas_data = Dataset.from_dict({
        "question": [user_input],
        "ground_truth": [quote_text],  # original quote
        "contexts": [[context]],
        "answer": [answer]
    })

    scores = {
        "faithfulness": round(faithfulness.compute(ragas_data), 3),
        "answer_relevancy": round(answer_relevancy.compute(ragas_data), 3),
        "context_precision": round(context_precision.compute(ragas_data), 3),
        "context_recall": round(context_recall.compute(ragas_data), 3)
    }

    result = {
        "Retrieved Quote": quote_text,
        "Author": author,
        "Tags": tags,
        "LLM Answer": answer,
        "RAGAS Scores": scores
    }

    return json.dumps(result, indent=2)


In [ ]:
gr.Interface(
    fn=query_rag_with_eval,
    inputs=gr.Textbox(label="Enter your natural language query"),
    outputs=gr.Textbox(label="RAG Answer & Evaluation"),
    title="RAG-Based Quote Retrieval with Mistral + RAGAS Evaluation",
    theme="soft"
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c26871c32c48e8c37.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Send request to OpenRouter API
data = {
    "model": "mistralai/mistral-7b-instruct",  # ✅ Use correct model name
    "messages": [{"role": "user", "content": prompt}]
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)

# Check for HTTP errors
if response.status_code != 200:
    return f"⚠️ API Error {response.status_code}: {response.text}"

response_json = response.json()

# Check if 'choices' is in the response
if 'choices' not in response_json:
    return f"⚠️ Unexpected response structure: {response_json}"

# Extract answer
answer = response_json['choices'][0]['message']['content']


NameError: name 'prompt' is not defined

In [ ]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import Dataset
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
import gradio as gr
import requests

# Load dataset
df = pd.read_json("hf://datasets/Abirate/english_quotes/quotes.jsonl", lines=True)

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create FAISS index
embeddings = model.encode(df['quote'].tolist(), show_progress_bar=True).astype('float32')
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Set OpenRouter key
API_KEY = "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"  # Replace this with your key
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

def query_rag_with_eval(user_input):
    try:
        # Step 1: Retrieve the most similar quote
        query_vec = model.encode([user_input]).astype("float32")
        D, I = index.search(query_vec, 1)
        quote = df.iloc[I[0][0]]
        quote_text = quote['quote']
        author = quote['author']
        tags = quote['tags']
        context = f"{quote_text} - {author} [{', '.join(tags)}]"

        # Step 2: Generate LLM Answer (OpenRouter - Mistral)
        prompt = f"""Answer the question using the quote below.\n\nQuestion: {user_input}\n\nContext: {context}\n\nAnswer:"""
        data = {
            "model": "mistral/mistral-7b-instruct",
            "messages": [{"role": "user", "content": prompt}]
        }

        response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
        answer = response.json()['choices'][0]['message']['content']

        # Step 3: RAGAS evaluation
        ragas_data = Dataset.from_dict({
            "question": [user_input],
            "ground_truth": [quote_text],
            "contexts": [[context]],
            "answer": [answer]
        })

        faith = round(faithfulness.compute(ragas_data), 3)
        relev = round(answer_relevancy.compute(ragas_data), 3)
        precision = round(context_precision.compute(ragas_data), 3)
        recall = round(context_recall.compute(ragas_data), 3)

        # Step 4: Format output
        result = f"""
 **Query**: {user_input}

 **Retrieved Quote**: "{quote_text}"
 **Author**: {author}
 **Tags**: {', '.join(tags)}

 **LLM Answer**: {answer}

 **RAGAS Evaluation**
- Faithfulness: {faith}
- Answer Relevancy: {relev}
- Context Precision: {precision}
- Context Recall: {recall}
        """
        return result.strip()

    except Exception as e:
        return f"⚠️ Error occurred: {str(e)}"

# Gradio Interface
gr.Interface(
    fn=query_rag_with_eval,
    inputs=gr.Textbox(label="Enter your natural language query", placeholder="e.g., show quotes about courage by women"),
    outputs=gr.Textbox(label="RAG Answer & Evaluation", lines=20),
    title="🧠 RAG-Based Semantic Quote Retrieval + RAGAS Evaluation",
    theme="soft"
).launch()


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

KeyboardInterrupt: 